In [1]:
# import library
from transformers import (AutoTokenizer,
                          RobertaForSequenceClassification,
                          Trainer,
                          TrainingArguments,)

from transformers import DataCollatorWithPadding
from datasets import load_dataset

In [2]:
model_id = 'vinai/phobert-base-v2'

# Initialising Tokeniser and Model
tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          cache_dir='./cache',
                                          model_max_length=256)

model = RobertaForSequenceClassification.from_pretrained(model_id,
                                                           num_labels=2,
                                                           cache_dir='./cache')

config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home4/bachpt/domain_classification/Data/Train/All_train_segmented.csv')
# df1 = df[df.isna().any(axis=1)]
# df1.info()
# df.notna().sum()

# rows_with_nan = df[df.isnull().any(axis=1)]
# print(rows_with_nan)

df.iloc[:,1].unique()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303675 entries, 0 to 303674
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   question  303675 non-null  object
 1   type      303675 non-null  object
dtypes: object(2)
memory usage: 4.6+ MB


In [22]:
# Define preprocess_function (tokenise)
def preprocess_function(examples):
    return tokenizer(examples['question'], truncation=True)

# Define Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Load data and apply tokeniser by mapping
ds = load_dataset('csv', data_files='/home4/bachpt/domain_classification/Data/Train/All_train_segmented.csv', split = 'train')
ds = ds.map(preprocess_function, batched=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/303675 [00:00<?, ? examples/s]

In [35]:
label2id = {"Chat": 0, "Uni": 1, "Others": 2}
ds_aligned = ds.map(lambda example: {"labels": label2id[example["type"]]})
# ds_aligned = ds.align_labels_with_mapping(label2id, "type")